# Code snippits os.fork, python stdlib mmap, numpy.memmap:

In [16]:
import os
import os.path as path
from tempfile import mkdtemp
import time
import mmap

import numpy as np

## fork - stack overflow
[StackOverflow how it works](https://stackoverflow.com/questions/33560802/pythonhow-os-fork-works) <br>

In [17]:

for i in range(2):
    print ('\n%i  ***********' %(i))
    pid = os.fork()
    if pid == 0:
        # We are in the child process.
        print ("%d (child) just was created by %d." % (os.getpid(), os.getppid()))
    else:
        # We are in the parent process.
        print ("%d (parent) just created %d." % (os.getpid(), pid))


0  ***********
1247 (parent) just created 1256.

1  ***********
1247 (parent) just created 1257.
1256 (child) just was created by 1247.

1  ***********1257 (child) just was created by 1247.

1256 (parent) just created 1258.
1258 (child) just was created by 1256.


[python docs (3.0)](https://docs.python.org/3.0/library/mmap.html) <br>
[python std lib codecs](https://docs.python.org/3/library/codecs.html) <br>
## python mmap :: memory map (char) large files
    * note that f.write takes text, mmap.(read&write) takes bytes object
    * mmap_obj displays newline chars, mmap_obj.decode('utf-8') executes them

In [40]:
# write a simple example file
with open("hello.txt", "w") as f:
    f.write("Hello Python!\n")

with open("hello.txt", "r+") as f:
    # memory-map the file, size 0 means whole file
    map = mmap.mmap(f.fileno(), 0)
    # read content via standard file methods
    
    s0 = map.readline()
    s = s0.decode('utf-8')
    s1 = bytes(s, 'utf-8')
    print('%30s: %s\n%30s: %s%30s: %s'%('raw read',s0, 'decoded', s, 'ReEncoded', s1))
    
    # read content via slice notation
    print(map[:5])  # prints "Hello"
    # update content using slice notation;
    # note that new content must have same size
    map[6:] = b" world!\n"
    # ... and read again using standard file methods
    map.seek(0)
    print(map.readline())  # prints "Hello  world!"
    # close the map
    map.close()

                      raw read: b'Hello Python!\n'
                       decoded: Hello Python!
                     ReEncoded: b'Hello Python!\n'
b'Hello'
b'Hello  world!\n'


[StackOverflow decoding advice](https://stackoverflow.com/questions/606191/convert-bytes-to-a-string) <br>

In [44]:
map = mmap.mmap(-1, 13)
map.write(b"Hello world!")

pid = os.fork()

if pid == 0: # In a child process
    map.seek(0)
    s0 = map.readline()
    print(s0)
    print(s0.decode('utf-8', 'backslashreplace'))
    print(s0.decode('cp437'))

    map.close()

b'Hello world!\x00'
Hello world! 
Hello world! 


## numpy memmap
[scipy docs numpy memmap](https://docs.scipy.org/doc/numpy/reference/generated/numpy.memmap.html) <br>
[memmap.flush](https://docs.scipy.org/doc/numpy/reference/generated/numpy.memmap.flush.html#numpy.memmap.flush) <br>
[StackOverflow Big Data with np.memmap](https://stackoverflow.com/questions/16149803/working-with-big-data-in-python-and-numpy-not-enough-ram-how-to-save-partial-r) <br>
```python
import numpy as np
a = np.memmap('test.mymemmap', dtype='float32', mode='w+', shape=(200000, 1000))
```

In [20]:
# import numpy as np

# from tempfile import mkdtemp
# import os.path as path

data = np.arange(12, dtype='float32')
data.resize((3,4))

filename = path.join(mkdtemp(), 'newfile.dat')
print('data:\n', data,'\n\nEmpty temporary file:\n%s\n'%(filename))

data:
 [[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]] 

Empty temporary file:
/var/folders/gf/ybz3wzn55m139k_4dw2v_tvm0000gn/T/tmpvvilnlmx/newfile.dat



#### Create a numpy memmap - With shape and type that matches the data

In [21]:
fp = np.memmap(filename, dtype='float32', mode='w+', shape=(3,4))
print(type(fp), '\n', fp, '\n\ndata\n',data)

<class 'numpy.memmap'> 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 

data
 [[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]


#### Write the data to the memmap array

In [22]:
fp[:] = data[:]
print('\nfp.filename == path.abspath(filename)', fp.filename == path.abspath(filename), '\tmode:', fp.mode)
fp


fp.filename == path.abspath(filename) True 	mode: w+


memmap([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]], dtype=float32)

#### delete the pointer, get a new one (to the temporary file)

In [23]:
del fp

newfp = np.memmap(filename, dtype='float32', mode='r', shape=(3,4))
newfp

memmap([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]], dtype=float32)

#### Get a new read-only pointer (with the correct size and type

In [24]:
fpr = np.memmap(filename, dtype='float32', mode='r', shape=(3,4))
fpr.flags.writeable

False

#### Get a new copy pointer (with the correct size and type
    * 2nd, 3rd & 4th cell below shows data is only written to mem
    * Original file pointer was mode='w+', fpr.mode: 'r', fpc.mode: 'c'

In [25]:
fpc = np.memmap(filename, dtype='float32', mode='c', shape=(3,4))
fpc.flags.writeable, fpc

(True, memmap([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]], dtype=float32))

In [26]:
fpc[0,:] = 0
fpc

memmap([[ 0.,  0.,  0.,  0.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]], dtype=float32)

In [27]:
fpc.flush()

In [28]:
fpr

memmap([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]], dtype=float32)

#### get pointer with offset

In [29]:
fpo = np.memmap(filename, dtype='float32', mode='r', offset=16)
fpo

memmap([ 4.,  5.,  6.,  7.,  8.,  9., 10., 11.], dtype=float32)

In [30]:
fpr.mode, fpc.mode, fpo.mode

('r', 'c', 'r')

In [45]:
import codecs
help(codecs)

Help on module codecs:

NAME
    codecs - codecs -- Python Codec Registry, API and helpers.

MODULE REFERENCE
    https://docs.python.org/3.7/library/codecs
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    
    Written by Marc-Andre Lemburg (mal@lemburg.com).
    
    (c) Copyright CNRI, All Rights Reserved. NO WARRANTY.

CLASSES
    builtins.object
        Codec
            StreamReader
            StreamWriter
        IncrementalDecoder
        IncrementalEncoder
        StreamReaderWriter
        StreamRecoder
    builtins.tuple(builtins.object)
        CodecInfo
    
    class Codec(builtins.object)
     |  Defines the interface for stateless encoders/decoders.
     |  
     |  The .encode()/.d